# **Hospital Emergency Department Dataset Cleaning & Preprocessing Pipeline**


In [4]:
import pandas as pd, numpy as np
from dateutil import parser as dparser
import re
pd.set_option("display.max_columns", 200)

In [5]:
df= pd.read_csv(r"C:\Users\shrav\OneDrive\Desktop\Projects\hospital_ed_overcrowding_analysis\hospital_ed_dirty_dataset.csv")

In [6]:
print("Rows:", len(df))
display(df.head(5))
print(df.isnull().sum())

Rows: 50800


,patient_id,arrival_timestamp,triage_level,chief_complaint,treatment_start_timestamp,discharge_timestamp,department,doctor_id,bed_id,wait_minutes,los_minutes
0,1,2024-01-26 11:11:00,2,Burn,11/04/2024 04:10,04-13-2024 04:00 PM,trauma dept,110.0,26.0,39.934283,28.000000
1,2,04-12-2024 01:49 PM,3,Trauma Injury,19/03/2024 15:02,2024-01-15 01:46:00,Pediatricks,133.0,126.0,42.953771,211.381791
2,3,2024-01-22 07:54,2,Breathing Difficulty,2024-01-03 04:35:00,2024-04-18 02:12,TRAUMA,115.0,51.0,25.316933,105.951783
3,4,2024/02/11 18:41,1,Dizzy,NaN,04-07-2024 04:38 AM,Pediatricks,145.0,154.0,61.584256,166.046084
4,5,2024-03-19 23:12,2,breathing difficulty,02-27-2024 03:06 PM,02-26-2024 10:25 PM,Emerg Med,107.0,38.0,20.610512,152.553603


patient_id                      0
arrival_timestamp               0
triage_level                    0
chief_complaint              3457
treatment_start_timestamp     970
discharge_timestamp          1563
department                   5084
doctor_id                    5053
bed_id                       5925
wait_minutes                    0
los_minutes                     0
dtype: int64


In [7]:
import numpy as np

df['_row_hash'] = df.astype(str).agg(''.join, axis=1)

duplicate_count = df.duplicated(subset=df.columns.tolist()).sum()
rowhash_dupes = df.duplicated(subset=['_row_hash']).sum()

print("Exact duplicate rows:", duplicate_count)
print("Row-hash duplicates:", rowhash_dupes)

df['audit_notes'] = ""


Exact duplicate rows: 800
Row-hash duplicates: 800


In [8]:
from dateutil import parser as dparser
import pandas as pd
import numpy as np

def parse_datetime_safe(val):
    if pd.isna(val):
        return pd.NaT
    s = str(val).strip()
    try:
        return pd.to_datetime(s, errors='raise')
    except:
        pass
    try:
        return dparser.parse(s, dayfirst=False, fuzzy=True)
    except:
        pass
    try:
        return dparser.parse(s, dayfirst=True, fuzzy=True)
    except:
        return pd.NaT


In [9]:
for col in ["arrival_timestamp", "treatment_start_timestamp", "discharge_timestamp"]:
    parsed_col = col + "_parsed"
    print(f"Parsing {col} → {parsed_col}")
    df[parsed_col] = df[col].apply(parse_datetime_safe)
    print(f"  Parsed valid: {df[parsed_col].notna().sum()} | NaT: {df[parsed_col].isna().sum()}\n")

df[[
    "arrival_timestamp", "arrival_timestamp_parsed",
    "treatment_start_timestamp", "treatment_start_timestamp_parsed"
]].head(10)


Parsing arrival_timestamp → arrival_timestamp_parsed


C:\Users\shrav\AppData\Local\Temp\ipykernel_8876\4138274285.py:10: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(s, errors='raise')


  Parsed valid: 50800 | NaT: 0

Parsing treatment_start_timestamp → treatment_start_timestamp_parsed


C:\Users\shrav\AppData\Local\Temp\ipykernel_8876\4138274285.py:10: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(s, errors='raise')


  Parsed valid: 49830 | NaT: 970

Parsing discharge_timestamp → discharge_timestamp_parsed


C:\Users\shrav\AppData\Local\Temp\ipykernel_8876\4138274285.py:10: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(s, errors='raise')


  Parsed valid: 49237 | NaT: 1563



,arrival_timestamp,arrival_timestamp_parsed,treatment_start_timestamp,treatment_start_timestamp_parsed
0,2024-01-26 11:11:00,2024-01-26 11:11:00,11/04/2024 04:10,2024-11-04 04:10:00
1,04-12-2024 01:49 PM,2024-04-12 13:49:00,19/03/2024 15:02,2024-03-19 15:02:00
2,2024-01-22 07:54,2024-01-22 07:54:00,2024-01-03 04:35:00,2024-01-03 04:35:00
3,2024/02/11 18:41,2024-02-11 18:41:00,NaN,NaT
4,2024-03-19 23:12,2024-03-19 23:12:00,02-27-2024 03:06 PM,2024-02-27 15:06:00
5,03-29-2024 10:45 PM,2024-03-29 22:45:00,2024/01/28 11:32,2024-01-28 11:32:00
6,2024/01/21 23:32,2024-01-21 23:32:00,2024/03/17 18:55,2024-03-17 18:55:00
7,2024-03-19 03:24,2024-03-19 03:24:00,2024-03-29 03:35:00,2024-03-29 03:35:00
8,2024-01-26 05:13,2024-01-26 05:13:00,2024-03-04 08:19:00,2024-03-04 08:19:00
9,28/04/2024 17:39,2024-04-28 17:39:00,02-09-2024 06:25 PM,2024-02-09 18:25:00


In [10]:
def norm(s):
    if pd.isna(s): 
        return np.nan
    s = str(s).lower().strip()
    s = re.sub(r'\s+', ' ', s)
    return s

df['chief_clean'] = df['chief_complaint'].apply(norm)
df['dept_clean'] = df['department'].apply(norm)

In [11]:
complaint_map = {
    'chest pain': ['chest pain','chestpain','chest pains','chest pain '],
    'breathing difficulty': ['breathing difficulty','breathing issues','breath difficulty'],
    'fever': ['fever','high fever','fever '],
    'headache': ['headache','head ache','headache '],
    'abdominal pain': ['abdominal pain','stomach pain','abd pain'],
    'trauma': ['trauma injury','trauma','injury - trauma'],
    'dizziness': ['dizziness','diziness','dizzy','dizzy '],
    'allergic reaction': ['allergic reaction','allergy','allergic '],
    'burn injury': ['burn injury','burn','burn ']
}

In [12]:
def map_val(v, m):
    if pd.isna(v): 
        return np.nan
    for canon, variants in m.items():
        for var in variants:
            if var in v:
                return canon
    return v

In [13]:
df['chief_mapped'] = df['chief_clean'].apply(lambda x: map_val(x, complaint_map))

dept_map = {
    'emergency medicine': ['emergency medicine','emergency','emerg med','em'],
    'trauma': ['trauma','trauma dept'],
    'cardiology': ['cardiology','cardio','cardio dept'],
    'pediatrics': ['pediatrics','peds','pediatricks','pediatric'],
    'general medicine': ['general medicine','gen med','medicine']
}

In [14]:
df['dept_mapped'] = df['dept_clean'].apply(lambda x: map_val(x, dept_map))

df[['chief_clean','chief_mapped','dept_clean','dept_mapped']].head(10)

,chief_clean,chief_mapped,dept_clean,dept_mapped
0,burn,burn injury,trauma dept,trauma
1,trauma injury,trauma,pediatricks,pediatrics
2,breathing difficulty,breathing difficulty,trauma,trauma
3,dizzy,dizziness,pediatricks,pediatrics
4,breathing difficulty,breathing difficulty,emerg med,emergency medicine
5,fever,fever,pediatric,pediatrics
6,head ache,headache,trauma,trauma
7,chest pain,chest pain,trauma dept,trauma
8,chest pains,chest pain,general medicine,general medicine
9,dizzy,dizziness,medicine,general medicine


In [15]:
df['triage'] = pd.to_numeric(df['triage_level'], errors='coerce')
df.loc[~df['triage'].between(1,5), 'triage'] = np.nan

df['doctor_id_clean'] = pd.to_numeric(df['doctor_id'], errors='coerce')
df['bed_id_clean'] = pd.to_numeric(df['bed_id'], errors='coerce')

df['wait_orig'] = pd.to_numeric(df['wait_minutes'], errors='coerce')
df['los_orig'] = pd.to_numeric(df['los_minutes'], errors='coerce')

In [16]:
df.loc[df['wait_orig'] < -120, 'wait_orig'] = np.nan
df.loc[df['wait_orig'] > 10000, 'wait_orig'] = np.nan
df.loc[df['los_orig'] < 0, 'los_orig'] = np.nan
df.loc[df['los_orig'] > 100000, 'los_orig'] = np.nan

df[['triage','doctor_id_clean','bed_id_clean','wait_orig','los_orig']].head(10)

,triage,doctor_id_clean,bed_id_clean,wait_orig,los_orig
0,2.0,110.0,26.0,39.934283,28.000000
1,3.0,133.0,126.0,42.953771,211.381791
2,2.0,115.0,51.0,25.316933,105.951783
3,1.0,145.0,154.0,61.584256,166.046084
4,2.0,107.0,38.0,20.610512,152.553603
5,1.0,NaN,158.0,20.731646,92.056215
6,4.0,NaN,69.0,34.839245,5.203185
7,4.0,NaN,56.0,-4.498357,86.262748
8,5.0,103.0,59.0,107.000000,138.854840
9,4.0,132.0,49.0,11.839518,35.261778


In [17]:
df['arrival_ts'] = df['arrival_timestamp_parsed']
df['treatment_ts'] = df['treatment_start_timestamp_parsed']
df['discharge_ts'] = df['discharge_timestamp_parsed']

df['wait_calc'] = (df['treatment_ts'] - df['arrival_ts']).dt.total_seconds() / 60
df['los_calc'] = (df['discharge_ts'] - df['treatment_ts']).dt.total_seconds() / 60

In [18]:
df.loc[df['wait_calc'].between(0, 10000), 'wait_clean'] = df['wait_calc'].round(1)
df.loc[df['los_calc'].between(0, 100000), 'los_clean'] = df['los_calc'].round(1)

df.loc[df['wait_clean'].isna() & df['wait_orig'].between(0,10000), 'wait_clean'] = df['wait_orig']
df.loc[df['los_clean'].isna() & df['los_orig'].between(0,100000), 'los_clean'] = df['los_orig']

df[['arrival_ts','treatment_ts','discharge_ts','wait_clean','los_clean']].head(10)

,arrival_ts,treatment_ts,discharge_ts,wait_clean,los_clean
0,2024-01-26 11:11:00,2024-11-04 04:10:00,2024-04-13 16:00:00,39.934283,28.000000
1,2024-04-12 13:49:00,2024-03-19 15:02:00,2024-01-15 01:46:00,42.953771,211.381791
2,2024-01-22 07:54:00,2024-01-03 04:35:00,2024-04-18 02:12:00,25.316933,105.951783
3,2024-02-11 18:41:00,NaT,2024-04-07 04:38:00,61.584256,166.046084
4,2024-03-19 23:12:00,2024-02-27 15:06:00,2024-02-26 22:25:00,20.610512,152.553603
5,2024-03-29 22:45:00,2024-01-28 11:32:00,2024-03-30 06:05:00,20.731646,88953.000000
6,2024-01-21 23:32:00,2024-03-17 18:55:00,2024-03-14 23:46:00,34.839245,5.203185
7,2024-03-19 03:24:00,2024-03-29 03:35:00,2024-01-27 08:34:00,NaN,86.262748
8,2024-01-26 05:13:00,2024-03-04 08:19:00,2024-04-23 00:46:00,107.000000,71547.000000
9,2024-04-28 17:39:00,2024-02-09 18:25:00,2024-03-31 22:03:00,11.839518,73658.000000


In [19]:
df['flag_treatment_before_arrival'] = (
    df['treatment_ts'].notna() & 
    df['arrival_ts'].notna() & 
    (df['treatment_ts'] < df['arrival_ts'])
)

df['flag_discharge_before_treatment'] = (
    df['discharge_ts'].notna() &
    df['treatment_ts'].notna() &
    (df['discharge_ts'] < df['treatment_ts'])
)

In [20]:
df.loc[df['flag_treatment_before_arrival'], ['wait_clean']] = np.nan
df.loc[df['flag_discharge_before_treatment'], ['los_clean']] = np.nan

mask_infer = (
    df['arrival_ts'].isna() &
    df['treatment_ts'].notna() &
    df['wait_clean'].notna()
)

In [21]:
df.loc[mask_infer, 'arrival_ts'] = (
    df.loc[mask_infer, 'treatment_ts'] - 
    pd.to_timedelta(df.loc[mask_infer, 'wait_clean'], unit='m')
)

df[['arrival_ts','treatment_ts','discharge_ts','wait_clean','los_clean',
    'flag_treatment_before_arrival','flag_discharge_before_treatment']].head(10)

,arrival_ts,treatment_ts,discharge_ts,wait_clean,los_clean,flag_treatment_before_arrival,flag_discharge_before_treatment
0,2024-01-26 11:11:00,2024-11-04 04:10:00,2024-04-13 16:00:00,39.934283,NaN,False,True
1,2024-04-12 13:49:00,2024-03-19 15:02:00,2024-01-15 01:46:00,NaN,NaN,True,True
2,2024-01-22 07:54:00,2024-01-03 04:35:00,2024-04-18 02:12:00,NaN,105.951783,True,False
3,2024-02-11 18:41:00,NaT,2024-04-07 04:38:00,61.584256,166.046084,False,False
4,2024-03-19 23:12:00,2024-02-27 15:06:00,2024-02-26 22:25:00,NaN,NaN,True,True
5,2024-03-29 22:45:00,2024-01-28 11:32:00,2024-03-30 06:05:00,NaN,88953.000000,True,False
6,2024-01-21 23:32:00,2024-03-17 18:55:00,2024-03-14 23:46:00,34.839245,NaN,False,True
7,2024-03-19 03:24:00,2024-03-29 03:35:00,2024-01-27 08:34:00,NaN,NaN,False,True
8,2024-01-26 05:13:00,2024-03-04 08:19:00,2024-04-23 00:46:00,107.000000,71547.000000,False,False
9,2024-04-28 17:39:00,2024-02-09 18:25:00,2024-03-31 22:03:00,NaN,73658.000000,True,False


In [22]:
before = len(df)
df = df.drop_duplicates()
after = len(df)

print("Exact duplicates removed:", before - after)

partial_dupes = df[df.duplicated(subset=['patient_id','arrival_ts'], keep=False)]
partial_dupes.shape

Exact duplicates removed: 800


(0, 34)

In [23]:
df['triage_final'] = df['triage']

df.loc[df['triage_final'].isna() & (df['wait_clean'] > 240), 'triage_final'] = 4

df['doctor_id_final'] = df['doctor_id_clean'].fillna(-1).astype(int)
df['bed_id_final'] = df['bed_id_clean'].fillna(-1).astype(int)

In [24]:
df['chief_final'] = df['chief_mapped'].fillna('unknown')
df['dept_final'] = df['dept_mapped'].fillna('unknown')

df[['triage_final','doctor_id_final','bed_id_final','chief_final','dept_final']].head(10)

,triage_final,doctor_id_final,bed_id_final,chief_final,dept_final
0,2.0,110,26,burn injury,trauma
1,3.0,133,126,trauma,pediatrics
2,2.0,115,51,breathing difficulty,trauma
3,1.0,145,154,dizziness,pediatrics
4,2.0,107,38,breathing difficulty,emergency medicine
5,1.0,-1,158,fever,pediatrics
6,4.0,-1,69,headache,trauma
7,4.0,-1,56,chest pain,trauma
8,5.0,103,59,chest pain,general medicine
9,4.0,132,49,dizziness,general medicine


In [25]:
df['arrival_date'] = df['arrival_ts'].dt.date
df['arrival_hour'] = df['arrival_ts'].dt.hour
df['arrival_dayofweek'] = df['arrival_ts'].dt.day_name()
df['is_weekend'] = df['arrival_ts'].dt.weekday >= 5

In [26]:
hourly = df.dropna(subset=['arrival_ts']).copy()
hourly['arrival_hour_bin'] = hourly['arrival_ts'].dt.floor('H')

arrivals = hourly.groupby('arrival_hour_bin').size().rename('arrivals')
discharges = hourly.groupby(hourly['discharge_ts'].dt.floor('H')).size().rename('discharges')

C:\Users\shrav\AppData\Local\Temp\ipykernel_8876\175879818.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly['arrival_hour_bin'] = hourly['arrival_ts'].dt.floor('H')
C:\Users\shrav\AppData\Local\Temp\ipykernel_8876\175879818.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  discharges = hourly.groupby(hourly['discharge_ts'].dt.floor('H')).size().rename('discharges')


In [27]:
hourly_summary = pd.concat([arrivals, discharges], axis=1).fillna(0)
hourly_summary['occupancy_est'] = hourly_summary['arrivals'].cumsum() - hourly_summary['discharges'].cumsum()

df[['arrival_date','arrival_hour','arrival_dayofweek','is_weekend']].head(10)


,arrival_date,arrival_hour,arrival_dayofweek,is_weekend
0,2024-01-26,11,Friday,False
1,2024-04-12,13,Friday,False
2,2024-01-22,7,Monday,False
3,2024-02-11,18,Sunday,True
4,2024-03-19,23,Tuesday,False
5,2024-03-29,22,Friday,False
6,2024-01-21,23,Sunday,True
7,2024-03-19,3,Tuesday,False
8,2024-01-26,5,Friday,False
9,2024-04-28,17,Sunday,True


In [28]:
df['flag_wait_outlier'] = (df['wait_clean'] > 1000)
df['flag_los_outlier'] = (df['los_clean'] > 10000)

df.loc[df['flag_wait_outlier'], 'wait_clean'] = np.nan
df.loc[df['flag_los_outlier'], 'los_clean'] = np.nan

df[['wait_clean','los_clean','flag_wait_outlier','flag_los_outlier']].head(10)

,wait_clean,los_clean,flag_wait_outlier,flag_los_outlier
0,39.934283,NaN,False,False
1,NaN,NaN,False,False
2,NaN,105.951783,False,False
3,61.584256,166.046084,False,False
4,NaN,NaN,False,False
5,NaN,NaN,False,True
6,34.839245,NaN,False,False
7,NaN,NaN,False,False
8,107.000000,NaN,False,True
9,NaN,NaN,False,True


In [29]:
df['wait_clean'] = df['wait_clean'].round(0).astype('Int64')
df['los_clean'] = df['los_clean'].round(0).astype('Int64')

In [30]:
df = df.rename(columns={
    'arrival_ts': 'arrival_time',
    'treatment_ts': 'treatment_start_time',
    'discharge_ts': 'discharge_time',
    'arrival_dayofweek': 'arrival_day_of_week',
    'triage_final': 'triage_level',
    'chief_final': 'chief_complaint',
    'dept_final': 'department',
    'doctor_id_final': 'doctor_id',
    'bed_id_final': 'bed_id',
    'wait_clean': 'wait_time_minutes',
    'los_clean': 'length_of_stay_minutes',
    'flag_treatment_before_arrival': 'flag_invalid_treatment_time',
    'flag_discharge_before_treatment': 'flag_invalid_discharge_time'
})

In [31]:
import pandas as pd
import numpy as np
from dateutil import parser

def safe_parse(x):
    try:
        return parser.parse(str(x))
    except:
        return None

In [32]:
df['arrival_time'] = df['arrival_time'].apply(safe_parse)
df['treatment_start_time'] = df['treatment_start_time'].apply(safe_parse)
df['discharge_time'] = df['discharge_time'].apply(safe_parse)

In [33]:
df['arrival_time'] = df['arrival_time'].dt.strftime("%H:%M %d-%m-%Y")
df['treatment_start_time'] = df['treatment_start_time'].dt.strftime("%H:%M %d-%m-%Y")
df['discharge_time'] = df['discharge_time'].dt.strftime("%H:%M %d-%m-%Y")

In [34]:
df['arrival_time'] = df['arrival_time'].fillna("")
df['treatment_start_time'] = df['treatment_start_time'].fillna("")
df['discharge_time'] = df['discharge_time'].fillna("")

In [35]:
df['arrival_date'] = pd.to_datetime(df['arrival_date'], errors='coerce').dt.strftime("%d-%m-%Y")

In [36]:
df[['arrival_time','treatment_start_time','discharge_time']].head(10)

,arrival_time,treatment_start_time,discharge_time
0,11:11 26-01-2024,04:10 04-11-2024,16:00 13-04-2024
1,13:49 12-04-2024,15:02 19-03-2024,01:46 15-01-2024
2,07:54 22-01-2024,04:35 03-01-2024,02:12 18-04-2024
3,18:41 11-02-2024,,04:38 07-04-2024
4,23:12 19-03-2024,15:06 27-02-2024,22:25 26-02-2024
5,22:45 29-03-2024,11:32 28-01-2024,06:05 30-03-2024
6,23:32 21-01-2024,18:55 17-03-2024,23:46 14-03-2024
7,03:24 19-03-2024,03:35 29-03-2024,08:34 27-01-2024
8,05:13 26-01-2024,08:19 04-03-2024,00:46 23-04-2024
9,17:39 28-04-2024,18:25 09-02-2024,22:03 31-03-2024


In [37]:
df

,patient_id,arrival_timestamp,triage_level,chief_complaint,treatment_start_timestamp,discharge_timestamp,department,doctor_id,bed_id,wait_minutes,los_minutes,_row_hash,audit_notes,arrival_timestamp_parsed,treatment_start_timestamp_parsed,discharge_timestamp_parsed,chief_clean,dept_clean,chief_mapped,dept_mapped,triage,doctor_id_clean,bed_id_clean,wait_orig,los_orig,arrival_time,treatment_start_time,discharge_time,wait_calc,los_calc,wait_time_minutes,length_of_stay_minutes,flag_invalid_treatment_time,flag_invalid_discharge_time,triage_level,doctor_id,bed_id,chief_complaint,department,arrival_date,arrival_hour,arrival_day_of_week,is_weekend,flag_wait_outlier,flag_los_outlier
0,1,2024-01-26 11:11:00,2,Burn,11/04/2024 04:10,04-13-2024 04:00 PM,trauma dept,110.0,26.0,39.934283,28.000000,12024-01-26 11:11:002Burn11/04/2024 04:1004-13...,,2024-01-26 11:11:00,2024-11-04 04:10:00,2024-04-13 16:00:00,burn,trauma dept,burn injury,trauma,2.0,110.0,26.0,39.934283,28.000000,11:11 26-01-2024,04:10 04-11-2024,16:00 13-04-2024,407099.0,-294490.0,40,<NA>,False,True,2.0,110,26,burn injury,trauma,26-01-2024,11,Friday,False,False,False
1,2,04-12-2024 01:49 PM,3,Trauma Injury,19/03/2024 15:02,2024-01-15 01:46:00,Pediatricks,133.0,126.0,42.953771,211.381791,204-12-2024 01:49 PM3Trauma Injury19/03/2024 1...,,2024-04-12 13:49:00,2024-03-19 15:02:00,2024-01-15 01:46:00,trauma injury,pediatricks,trauma,pediatrics,3.0,133.0,126.0,42.953771,211.381791,13:49 12-04-2024,15:02 19-03-2024,01:46 15-01-2024,-34487.0,-92956.0,<NA>,<NA>,True,True,3.0,133,126,trauma,pediatrics,12-04-2024,13,Friday,False,False,False
2,3,2024-01-22 07:54,2,Breathing Difficulty,2024-01-03 04:35:00,2024-04-18 02:12,TRAUMA,115.0,51.0,25.316933,105.951783,32024-01-22 07:542Breathing Difficulty2024-01-...,,2024-01-22 07:54:00,2024-01-03 04:35:00,2024-04-18 02:12:00,breathing difficulty,trauma,breathing difficulty,trauma,2.0,115.0,51.0,25.316933,105.951783,07:54 22-01-2024,04:35 03-01-2024,02:12 18-04-2024,-27559.0,152497.0,<NA>,106,True,False,2.0,115,51,breathing difficulty,trauma,22-01-2024,7,Monday,False,False,False
3,4,2024/02/11 18:41,1,Dizzy,NaN,04-07-2024 04:38 AM,Pediatricks,145.0,154.0,61.584256,166.046084,42024/02/11 18:411Dizzy nan04-07-2024 04:38 AM...,,2024-02-11 18:41:00,NaT,2024-04-07 04:38:00,dizzy,pediatricks,dizziness,pediatrics,1.0,145.0,154.0,61.584256,166.046084,18:41 11-02-2024,,04:38 07-04-2024,NaN,NaN,62,166,False,False,1.0,145,154,dizziness,pediatrics,11-02-2024,18,Sunday,True,False,False
4,5,2024-03-19 23:12,2,breathing difficulty,02-27-2024 03:06 PM,02-26-2024 10:25 PM,Emerg Med,107.0,38.0,20.610512,152.553603,52024-03-19 23:122breathing difficulty02-27-20...,,2024-03-19 23:12:00,2024-02-27 15:06:00,2024-02-26 22:25:00,breathing difficulty,emerg med,breathing difficulty,emergency medicine,2.0,107.0,38.0,20.610512,152.553603,23:12 19-03-2024,15:06 27-02-2024,22:25 26-02-2024,-30726.0,-1001.0,<NA>,<NA>,True,True,2.0,107,38,breathing difficulty,emergency medicine,19-03-2024,23,Tuesday,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,29/03/2024 21:07,3,trauma injury,2024-02-16 02:38,04-16-2024 05:45 AM,NaN,146.0,74.0,42.878290,169.555526,4999629/03/2024 21:073trauma injury2024-02-16 ...,,2024-03-29 21:07:00,2024-02-16 02:38:00,2024-04-16 05:45:00,trauma injury,NaN,trauma,NaN,3.0,146.0,74.0,42.878290,169.555526,21:07 29-03-2024,02:38 16-02-2024,05:45 16-04-2024,-61589.0,86587.0,<NA>,<NA>,True,False,3.0,146,74,trauma,unknown,29-03-2024,21,Friday,False,False,True
49996,49997,29/02/2024 03:12,3,breathing difficulty,04-24-2024 10:08 AM,2024/04/12 01:16,CARDIO,146.0,105.0,25.732604,132.852483,4999729/02/2024 03:123breathing difficulty04-2...,,2024-02-29 03:12:00,2024-04-24 10:08:00,2024-04-12 01:16:00,breathing difficulty,cardio,breathing difficulty,cardiology,3.0,146.0,105.0,25.732604,132.852483,03:12 29-02-2

In [39]:
[x for x in df.columns if df.columns.tolist().count(x) > 1]


['triage_level',
 'chief_complaint',
 'department',
 'doctor_id',
 'bed_id',
 'triage_level',
 'doctor_id',
 'bed_id',
 'chief_complaint',
 'department']

In [40]:
df = df.loc[:, ~df.columns.duplicated()].copy()


In [41]:
df = df[final_cols].copy()


In [42]:
df.columns


Index(['patient_id', 'arrival_time', 'treatment_start_time', 'discharge_time',
       'arrival_date', 'arrival_hour', 'arrival_day_of_week', 'is_weekend',
       'triage_level', 'chief_complaint', 'department', 'doctor_id', 'bed_id',
       'wait_time_minutes', 'length_of_stay_minutes',
       'flag_invalid_treatment_time', 'flag_invalid_discharge_time',
       'flag_wait_outlier', 'flag_los_outlier'],
      dtype='object')

In [47]:
df['chief_clean'] = (
    df['chief_complaint']
    .str.strip()
    .str.lower()
    .str.replace(r'[^a-zA-Z\s]', '', regex=True)
)

replace_chief = {
    'head ache': 'headache',
    'dizzy': 'dizziness',
    'chest pains': 'chest pain',
    'breath difficulty': 'breathing difficulty',
    'breathing diffculty': 'breathing difficulty'
}

df['chief_clean'] = df['chief_clean'].replace(replace_chief, regex=False)
df['chief_clean'] = df['chief_clean'].str.replace(' +',' ', regex=True).str.strip()
df['chief_clean'] = df['chief_clean'].fillna('unknown')


In [48]:
chief_map = {
    'burn': 'burn injury',
    'burn injury': 'burn injury',
    'headache': 'headache',
    'chest pain': 'chest pain',
    'breathing difficulty': 'breathing difficulty',
    'fever': 'fever',
    'dizziness': 'dizziness',
    'injury trauma': 'trauma',
    'trauma injury': 'trauma',
    'trauma': 'trauma'
}

df['chief_final'] = df['chief_clean'].map(chief_map).fillna(df['chief_clean'])


In [49]:
df['dept_clean'] = (
    df['department']
    .str.strip()
    .str.lower()
    .str.replace(r'[^a-zA-Z\s]', '', regex=True)
)

replace_dept = {
    'pediatricks': 'pediatrics',
    'pediatric': 'pediatrics',
    'peds': 'pediatrics',
    'emerg med': 'emergency medicine',
    'general medicine': 'general medicine',
    'medicine': 'general medicine',
    'cardio': 'cardiology',
    'trauma dept': 'trauma'
}

df['dept_final'] = df['dept_clean'].replace(replace_dept, regex=False)
df['dept_final'] = df['dept_final'].fillna('unknown')


In [50]:
df['doctor_id'] = pd.to_numeric(df['doctor_id'], errors='coerce').fillna(-1).astype(int)
df['bed_id'] = pd.to_numeric(df['bed_id'], errors='coerce').fillna(-1).astype(int)


In [51]:
df['chief_complaint'] = df['chief_final']
df['department'] = df['dept_final']


In [52]:
df.drop(columns=[
    'chief_clean','chief_final',
    'dept_clean','dept_final'
], errors='ignore', inplace=True)


In [53]:
df[['chief_complaint','department','doctor_id','bed_id']].head(20)


,chief_complaint,department,doctor_id,bed_id
0,burn injury,trauma,110,26
1,trauma,pediatrics,133,126
2,breathing difficulty,trauma,115,51
3,dizziness,pediatrics,145,154
4,breathing difficulty,emergency medicine,107,38
5,fever,pediatrics,-1,158
6,headache,trauma,-1,69
7,chest pain,trauma,-1,56
8,chest pain,general medicine,103,59
9,dizziness,general medicine,132,49


In [54]:
final_cols = [
    'patient_id',
    'arrival_time','treatment_start_time','discharge_time',
    'arrival_date','arrival_hour','arrival_day_of_week','is_weekend',
    'triage_level','chief_complaint','department',
    'doctor_id','bed_id',
    'wait_time_minutes','length_of_stay_minutes',
    'flag_invalid_treatment_time','flag_invalid_discharge_time',
    'flag_wait_outlier','flag_los_outlier'
]

final_cols = [c for c in final_cols if c in df.columns]

clean_full_path = r"C:\Users\shrav\OneDrive\Desktop\Projects\hospital_ed_overcrowding_analysis\hospital_ed_cleaned_full.csv"

df[final_cols].to_csv(clean_full_path, index=False)

clean_full_path



'C:\\Users\\shrav\\OneDrive\\Desktop\\Projects\\hospital_ed_overcrowding_analysis\\hospital_ed_cleaned_full.csv'

In [2]:
df= pd.read_csv(r"C:\Users\shrav\OneDrive\Desktop\Projects\hospital_ed_overcrowding_analysis\hospital_ed_cleaned_full.csv")

In [55]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd

# MySQL credentials
username = "root"
password = "Shvetang@24"
host = "localhost"
port = "3306"
database = "hospital_ed"   # <-- NEW DATABASE

# Encode password
password_encoded = quote_plus(password)

# Create SQLAlchemy engine (server-level)
engine = create_engine(f"mysql+pymysql://{username}:{password_encoded}@{host}:{port}/")

# Create engine for your specific database
engine_db = create_engine(f"mysql+pymysql://{username}:{password_encoded}@{host}:{port}/{database}")
table_name = "ed_visits"
df.to_sql(table_name, engine_db, if_exists="replace", index=False)
print("Data uploaded successfully!")


Data uploaded successfully!
